In [1]:
#GAN
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np

# Load and preprocess CIFAR-10 dataset
(x_train, _), (_, _) = cifar10.load_data()
x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # Normalize pixel values to [-1, 1]

# Define generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(4 * 4 * 256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh'))
    return model

# Define discriminator model
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=(32, 32, 3)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# Define GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Define hyperparameters
latent_dim = 100
epochs = 2
batch_size = 128
half_batch = batch_size // 2

# Build and compile discriminator
discriminator = build_discriminator()
discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

# Build and compile generator
generator = build_generator(latent_dim)
generator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

# Build and compile GAN model
gan = build_gan(generator, discriminator)
gan.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

# Training loop
for epoch in range(epochs):
    for _ in range(len(x_train) // batch_size):
        # Train discriminator
        idx = np.random.randint(0, x_train.shape[0], half_batch)
        real_images = x_train[idx]
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_images = generator.predict(noise)
        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)
        
        # Print progress
        print(f"Epoch {epoch + 1}/{epochs}, Batch {_}/{len(x_train) // batch_size}, D_loss={d_loss[0]}, G_loss={g_loss}")

c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1/5, Batch 0/781, D_loss=0.679856538772583, G_loss=0.6925722360610962
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Epoch 1/5, Batch 1/781, D_loss=0.6827269196510315, G_loss=0.6917506456375122
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Epoch 1/5, Batch 2/781, D_loss=0.682643711566925, G_loss=0.6908720135688782
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Epoch 1/5, Batch 3/781, D_loss=0.6834644079208374, G_loss=0.6897599697113037
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Epoch 1/5, Batch 4/781, D_loss=0.6838563680648804, G_loss=0.6884673833847046
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Epoch 1/5, Batch 5/781, D_loss=0.6845521926879883, G_loss=0.6868279576301575
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Epoch 1/5, Batch 6/781, D_loss=0.6859678030014038, G_loss=0.6848310232162476
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Epoch 1/5, Batch 7/781, D_loss=0.6877325773239136, G_loss=0.6824524998664856
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Epoch 1/5, Batch 8/781, D_loss=0.6891524791717529, G_loss=0.6794276833534241
1/1 ━